# Cosserat equation in mixed form

In this tutorial we present how to solve the Cosserat equation with [PyGeoN](https://github.com/compgeo-mox/pygeon) in mixed form.  The unknowns are the stress $\sigma$, the micro-stress $\omega$, the displacement $u$, and the rotation $r$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given the macroscopic 
$\lambda_\sigma$ Lamé constant and $\mu_\sigma$ the Kirchhoff modulus and their microscopic version $\lambda_\omega$ and $\mu_\omega$, 
we want to solve the following problem: find $(\sigma, \omega, u, r)$ such that
\begin{align*}
&A_\sigma \sigma - \nabla u + {\rm asym}^* r = 0\\
&A_\omega \omega - \nabla r = 0\\
&\nabla \cdot \sigma = - b\\
&{\rm asym}\, \sigma - \nabla \cdot \omega = 0
\end{align*}
with $b$ a body force, which is set to $0$ in this example.
The operator $A_\sigma$ is defined as
$$
A_\sigma \sigma = \frac1{2\mu_\sigma} \left( \mathrm{sym} \sigma - \frac{\lambda_\sigma}{2\mu_\sigma + d \lambda_\sigma} \mathrm{Tr} (\sigma) I\right) + \frac1{2\mu_\sigma^c} \mathrm{skw} \sigma
$$
with $d$ the dimension of the domain, here $d=2$. While the operator $A_\omega$ depends on the dimension 
and is defined as 
$$
A_\omega \omega = \frac1{2\mu_\omega} \left( \mathrm{sym} \omega - \frac{\lambda_\omega}{2\mu_\omega + d \lambda_\omega} \mathrm{Tr} (\omega) I\right) + \frac1{2\mu_\omega^c} \mathrm{skw} \omega
 \qquad d = 3
$$
$$
A_\omega \omega = \frac{1}{2\mu_\omega}\omega  \qquad d = 2
$$
Finally, ${\rm asym}$ is the asymmetric operator, which is defined as followed
$$
    {\rm asym }\, \sigma = \begin{bmatrix}
        \sigma_{32} - \sigma_{23} \\
        \sigma_{13} - \sigma_{31} \\
        \sigma_{21} - \sigma_{12}
    \end{bmatrix} 
    \qquad
    {\rm asym}^* r =
    \begin{bmatrix}
        0    & -r_3 & r_2  \\
        r_3  & 0    & -r_1 \\
        -r_2 & r_1  & 0
    \end{bmatrix}
    \qquad
    d = 3
$$
$$
    {\rm asym} \, \sigma  = \sigma_{21} - \sigma_{12} \qquad
    {\rm asym}^* r =
    \begin{bmatrix}
        0 & -r \\
        r & 0
    \end{bmatrix} \qquad
    d = 2.
$$
For this test case we set the following boundary conditions related to the so-called footstep problem:
$$ u = 0 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot \sigma = [0, 0]^\top \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \qquad \nu \cdot \sigma = [0, -1e-3]^\top \text{ on } \partial_{top} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [ ]:
import os

import numpy as np
import porepy as pp
import scipy.sparse as sps

import pygeon as pg

We create now the grid, since we use vector RT0 for ${\sigma}$, RT0 for $\omega$, vector piece-wise constant for $u$ and piece-wise constant for $r$, we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d with the only attention that in 3d the micro-stress is a vector RT0 and the rotations is a vector piece-wise constant.

In [ ]:
mesh_size = 0.1
dim = 2

sd = pg.unit_grid(dim, mesh_size, as_mdg=False)
sd.compute_geometry()

We define now the finite element spaces for all the variables.

In [ ]:
key = "cosserat"

# finite element spaces
vec_rt0 = pg.VecRT0(key)
rt0 = pg.RT0(key)
vec_p0 = pg.VecPwConstants(key)
p0 = pg.PwConstants(key)

# build the degrees of freedom
dofs = np.array([vec_rt0.ndof(sd), rt0.ndof(sd), vec_p0.ndof(sd), p0.ndof(sd)])

With the following code we set the data, in particular the Lamé and the Kirchhoff modulus, and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [ ]:
# data for sigma
param_sigma = {pg.LAME_MU: 0.5, pg.LAME_LAMBDA: 1, pg.LAME_MU_COSSERAT: 0.25}
data_sigma = pp.initialize_data({}, key, param_sigma)


# data for omega
sot = pp.SecondOrderTensor(np.ones(sd.num_cells))
data_omega = pp.initialize_data({}, key, {pg.SECOND_ORDER_TENSOR: sot})

# select the portions of the boundary where we want to apply the boundary conditions
bottom = np.isclose(sd.face_centers[1, :], 0)
top = np.isclose(sd.face_centers[1, :], 1)
left = np.isclose(sd.face_centers[0, :], 0)
right = np.isclose(sd.face_centers[0, :], 1)

# select the part of the boundary where we apply the essential boundary conditions
b_faces = np.logical_or.reduce((top, left, right))
ess_dof = np.tile(b_faces, sd.dim)

# function for the essential boundary conditions
val = np.array([[0, 0, 0], [0, -1e-3, 0]])
fct = lambda pt: val if np.isclose(pt[1], 1) else 0 * val

# interpolate the essential boundary conditions
ess_val = vec_rt0.interpolate(sd, fct)

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [ ]:
# assemble mass matrices
Ms = vec_rt0.assemble_mass_matrix_cosserat(sd, data_sigma)
Mw = rt0.assemble_mass_matrix(sd, data_omega)
Mu = vec_p0.assemble_mass_matrix(sd)
Mr = p0.assemble_mass_matrix(sd)

# assemble matrices associated with the divergence and asymmetry operators
div_s = Mu @ vec_rt0.assemble_diff_matrix(sd)
asym = Mr @ vec_rt0.assemble_asym_matrix(sd, as_pwconstant=True)
div_w = Mr @ rt0.assemble_diff_matrix(sd)

# assemble the saddle point problem matrix
A = sps.block_diag([Ms, Mw])
B = sps.block_array(
    [
        [-div_s, None],
        [asym, -div_w],
    ],
    format="csc",
)

spp = sps.block_array(
    [
        [A, -B.T],
        [B, None],
    ],
    format="csc",
)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (stress boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [ ]:
# solve the linear system
ls = pg.LinearSystem(spp, np.zeros(spp.shape[0]))
ls.flag_ess_bc(np.where(ess_dof)[0], ess_val)
x = ls.solve()

# split the solution into the components
idx = np.cumsum(dofs[:-1])
sigma, omega, u, r = np.split(x, idx)

We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [ ]:
# post process of the displacement
proj_u = vec_p0.eval_at_cell_centers(sd)
cell_u = (proj_u @ u).reshape((3, -1))

# post process of the rotation
proj_r = p0.eval_at_cell_centers(sd)
cell_r = proj_r @ r

# export the solution
folder_name = os.path.join(os.getcwd(), key)
file_name = "sol"

save = pp.Exporter(sd, file_name, folder_name=folder_name)
save.write_vtu([("cell_u", cell_u), ("cell_r", cell_r)])

A representation of the computed solution is given below.

<img src="./fig/cosserat.png"/>

In [ ]:
# Consistency check
assert np.isclose(np.linalg.norm(sigma), 0.0013947854064159603)
assert np.isclose(np.linalg.norm(omega), 1.1443428299962513e-05)
assert np.isclose(np.linalg.norm(u), 2.403520428780552e-05)
assert np.isclose(np.linalg.norm(r), 5.636222917225164e-08)